# Motion Saliency

In [1]:
"""
Compute Motion Saliency

Here we take a video and iterate through each frame to get the motion saliency
The computation is based on OpenCV fonction based on Wang and Dudek’s 2014 publication
For each frame, we get an array [width x heigh] were each value represent 0=Non-salient & 1=salient
It is mostly based on detection of the background vs object

It is possible to display the video saliency at the same time as the computing. 

This function return an array of (nframe x width x heigh).

"""

import cv2 
import numpy as np
import progressbar

def ComputeMotionSaliency(path, path_save, file , frameID=None, show=False):
    
    # import the video and settings
    video = cv2.VideoCapture(path+file)
    saliency = None
    results = None
    frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    bar = progressbar.ProgressBar(max_value=frames);
    i=1
    
    
    # Iterate trough frames and get motion saliency per frame    
    while (video.isOpened()): 
        # Read frame
        ret, frame = video.read() 

        # If there is no frame (end of video), then stop
        if not ret:
            break

        # Set up if first frame
        if saliency is None:
            saliency = cv2.saliency.MotionSaliencyBinWangApr2014_create() # Motion Saliency
            saliency.setImagesize(frame.shape[1], frame.shape[0])
            saliency.init()

        # Use Wang method for motion saliency
        grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert to grey scale
        (success, saliencyMap) = saliency.computeSaliency(grayFrame) # compute saliency

        if results is None:
            results = np.atleast_3d(saliencyMap)
        else:
            results = np.append(results, np.atleast_3d(saliencyMap), axis=2)

        
        if i == frameID:
            saliencyMapColored = (saliencyMap * 255).astype("uint8")
            cv2.imwrite(path_save+"{}_frame{}_motionSalience.jpg".format(file[:-4], frameID), saliencyMapColored)
            print('saved')
            
        
        if show:
            saliencyMapColored = (saliencyMap * 255).astype("uint8") # Transform to color values
            cv2.imshow("Map", saliencyMapColored) # Display the frame
            key = cv2.waitKey(1) & 0xFF
        
        # Update Bar
        bar.update(i)
        i+=1
    
    cv2.destroyAllWindows()
    return results.swapaxes(0,2).swapaxes(1,2)

In [ ]:
cv2.destroyAllWindows()
result = ComputeMotionSaliency(path, path_save, file , frameID=100, show=True);

--------------
# Static Saliency 

In [2]:
def getVideoValues(path):
    
    video = cv2.VideoCapture(path)
    
    if video.isOpened(): 
        
        width  = video.get(cv2.CAP_PROP_FRAME_WIDTH)   # float
        height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
        fps = video.get(cv2.CAP_PROP_FPS)
        frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        
    return video, frames, (width, height), fps

In [3]:
import cv2 
import numpy as np
import progressbar

"""
Compute Static Saliency

Here we take a video and iterate through each frame to get the static saliency of each frame
The computation is based on OpenCV fonction based on Hou and Zhang in their 2007 CVPR paper
For each frame, we get an array [width x heigh] were each value represent 0=Non-salient & 1=salient

This function return an array of (nframe x width x heigh).

"""

def ComputeStaticSaliency(path, file, method=None, show=False):
    
    # import the video and settings
    video, frames, size, fps = getVideoValues(path+file)
    
    results = None
    bar = progressbar.ProgressBar(max_value=frames);
    i=1
    
    # Set saliency algorithm to use
    if method is 'spectral':  
#         print('Using Spectral Residual Method')
        saliency =  cv2.saliency.StaticSaliencySpectralResidual_create() # Use Spectral Residual method
    else: 
#         print('Using Fine Grained Method')
        saliency = cv2.saliency.StaticSaliencyFineGrained_create() # Use Fine Grained method 

    
    # Iterate trough frames and get saliency per frame    
    while (video.isOpened()): 
        # Read frame
        ret, frame = video.read() 

        # If there is no frame (end of video), then stop
        if not ret:
            break

        # Apply chosen method on frame
        (success, saliencyMap) = saliency.computeSaliency(frame)
        
        # Save result in np array
        if results is None:
            results = np.atleast_3d(saliencyMap)
        else:
            results = np.append(results, np.atleast_3d(saliencyMap), axis=2)
            
            
        # colored and save new frame in video
        saliencyMapColored = (saliencyMap * 255).astype("uint8") # change [0 to 1] to [1 to 255] to display
        
        # Display result
        if show:
            cv2.imshow("{} Map".format(method), saliencyMapColored) # Show frame / update new frame
            key = cv2.waitKey(1) & 0xFF
            
        # Update Bar
        bar.update(i)
        i+=1
    
    cv2.destroyAllWindows()
    return results.swapaxes(0,2).swapaxes(1,2)

In [ ]:
path = "C:/Users/ramme/Dev/Manip-Maitrise/Z_BanqueVideo/"
path_save = "C:/Users/ramme/Dev/Analyse-Maitrise/Videos/result/OpenCV/"
files = ['1Neutral_escalade.mp4', '1Neutral_kid_writing.mp4', '2vid18.mp4', '2vid44.mp4', '3vid9.mp4', '3vid120.mp4']
frames = [300, 300, 175, 175, 175, 350]
i=0

for file, frame in zip(files, frames):
    result = ComputeStaticSaliency(path, file, method='FineGrained', show=True);

-----------------------------------
# Motion Detection

In [4]:
import cv2
import numpy as np
import progressbar

def ComputeMotionDetection(path, path_save, file, frameID=None, treshold_value=20, show=False):
    
    # import the video and settings
    video, frames, size, fps = getVideoValues(path+file)
    results = None
    bar = progressbar.ProgressBar(max_value=frames);
    i=1
    
    ret, frame1 = video.read()
    ret, frame2 = video.read()
    
    # Iterate trough frames and get saliency per frame    
    while (video.isOpened()): 
                    
        # Difference between Frame 1 and Frame 2
        diff = cv2.absdiff(frame1, frame2)
        
        # Convert to gray Scale
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
            
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        
        _, thresh = cv2.threshold(blur, treshold_value, 255, cv2.THRESH_BINARY)
        
        if show:
            cv2.imshow('Difference Frame', thresh)
            key = cv2.waitKey(1) & 0xFF
        
        # save frame in JPG
        if i==frameID:
            cv2.imwrite(path_save+"{}_frame{}_MotionDetection{}.jpg".format(file[:-4], frameID, treshold_value), thresh)
        
        # Save result in np array
        if results is None:
            results = np.atleast_3d(thresh)
        else:
            results = np.append(results, np.atleast_3d(thresh), axis=2)
            
        
        # Assign frame2(image) to frame1(image)
        frame1 = frame2

        #Read new frame2
        ret, frame2 = video.read()
        
        # Update Bar
        bar.update(i)
        i+=1
                
        if not ret:
            break
            
    cv2.destroyAllWindows()
    return results.swapaxes(0,2).swapaxes(1,2)

In [ ]:
path = "C:/Users/ramme/Dev/Manip-Maitrise/Z_BanqueVideo/"
path_save = "C:/Users/ramme/Dev/Analyse-Maitrise/Videos/result/OpenCV/"
files = ['1Neutral_escalade.mp4', '1Neutral_kid_writing.mp4', '2vid18.mp4', '2vid44.mp4', '3vid9.mp4', '3vid120.mp4']
frames = [300, 300, 175, 175, 175, 350]
i=0

for file, frame in zip(files, frames):
    result = ComputeMotionDetection(path, path_save, file , frame, 50 ,show=True);

# Extract Specific Frame (Static Salience)

In [5]:
import cv2
import numpy as np 
import progressbar

def ComputeSpecificFrame(path, path_save, file , frameID=100, show=False):
      
    # import the video and settings
    video = cv2.VideoCapture(path+file)
    frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if frameID > frames:
        print('Frame do not exist, please select a frame under {}'.format(frames))
    
    i=1
    
    # Iterate trough frames and get saliency per frame    
    while (video.isOpened()): 
        ret, frame = video.read() 

        # If there is no frame (end of video), then stop
        if not ret:
            break
            
        elif i == frameID:
            
            # Save Original Frame
            cv2.imwrite(path_save+"{}_frame{}_original.jpg".format(file[:-4], frameID), frame)
            
            # Compute SpectralResidualfor static Saliency
            saliency_SR =  cv2.saliency.StaticSaliencySpectralResidual_create()
            (success, saliencyMap_SR) = saliency_SR.computeSaliency(frame)
            saliencyMap_SR = (saliencyMap_SR * 255).astype("uint8")
            cv2.imwrite(path_save+"{}_frame{}_SpectralResidual.jpg".format(file[:-4], frameID), saliencyMap_SR)

            # Compute Fine Grained 
            saliency_FG = cv2.saliency.StaticSaliencyFineGrained_create()
            (success, saliencyMap_FG) = saliency_FG.computeSaliency(frame)
            saliencyMap_FG = (saliencyMap_FG * 255).astype("uint8")
            cv2.imwrite(path_save+"{}_frame{}_FineGrained.jpg".format(file[:-4], frameID), saliencyMap_FG)
            

            
            if show:
                cv2.imshow("Real Image", frame)
                cv2.imshow("Spectral Residual", saliencyMap_SR)
                cv2.imshow("Fine Grained Map", saliencyMap_FG)
                
                cv2.waitKey()
                cv2.destroyAllWindows()
        
        i+=1
        
    return

In [ ]:
path = "C:/Users/ramme/Dev/Manip-Maitrise/Z_BanqueVideo/"
path_save = "C:/Users/ramme/Dev/Analyse-Maitrise/Videos/result/OpenCV/"
files = ['1Neutral_escalade .mp4', '1Neutral_kid_writing.mp4', '2vid18.mp4', '2vid44.mp4', '3vid9.mp4', '3vid120.mp4']
frames = [300, 300, 175, 175, 175, 350]
i=0

for file, frame in zip(files, frames):
    result = ComputeSpecificFrame(path, path_save, file, frame, show=True);
    result = ComputeMotionSaliency(path, path_save, file , frame, show=False);

# Analyses

In [ ]:
import os
cv2.destroyAllWindows()

# Setup
path = "C:/Users/ramme/Dev/Manip-Maitrise/Z_BanqueVideo/"
path_save = "C:/Users/ramme/Dev/Analyse-Maitrise/Videos/result/OpenCV_NPZ/"



# Select file to use
# files = ['1Neutral_escalade.mp4', '2vid44.mp4'] # test file
files = os.listdir(path) # all data


# Bar update
for file in files:
    print('Computing:', file)
    
    data_method= []
    data_array = []
    
    # Compute Fine Grained Motion
    data_method.append('FineGrained')
    data_array.append(ComputeStaticSaliency(path, file, 'FineGrained'))
    
    # Compute Motion Detection
    data_method.append('MotionDetection')
    data_array.append(ComputeMotionDetection(path, path_save, file))
    
    # Compute Motion Saliency
    data_method.append('MotionSaliency')
    data_array.append(ComputeMotionSaliency(path, path_save, file))
       
    
    np.savez(path_save+'{}_computeNPZ.npz'.format(file), data_method=data_method, data_array=data_array)  # data is a dict here

In [ ]:
data = np.load(path_save+'1EDIT_neutre_train_a.mp4_computeNPZ.npz', allow_pickle=True)
data.files
d=data['data_method']
print(d[0].shape)
print(d[1].shape)

# Split Video In Time & Space

In [6]:
import numpy as np
import pandas as pd
import os


# files = ['1EDIT_neutre_train_a.mp4_computeNPZ.npz', '1Neutral_baseball_GA.mp4_computeNPZ.npz'] # test file
files = os.listdir('./result/OpenCV_NPZ/') # all data

def mean_time_space(time_clusters=1, space_clusters_x=1, space_clusters_y=1):
    
    df = pd.DataFrame()

    # Bar update
    for file in files:
        # Load Completed File  
        try:
            data = np.load('./result/OpenCV_NPZ/'+file, allow_pickle=True)
        except:
            print(file, 'ERROR: not working for some reason')
            continue

        for selected_method in [0,1]:
            d=data['data_array'][selected_method]
            
            row = pd.DataFrame.from_dict({
                    'method':[data['data_method'][selected_method]],
                    'video':[file[:-15]]})

            time_bins      = int(d.shape[0]/time_clusters)
            space_bins_y   = int(d.shape[1]/space_clusters_x)
            space_bins_x   = int(d.shape[2]/space_clusters_y)

            # Split in time / space:
            for time in range(time_clusters):
                for space_x in range(space_clusters_x):
                    for space_y in range(space_clusters_y):
                        row['T'+str(time)+'X'+str(space_x)+'Y'+str(space_y)]= d[time*time_bins:(time+1)*time_bins,space_y*space_bins_y:(space_y+1)*space_bins_y, space_x*space_bins_x:(space_x+1)*space_bins_x].mean() 
            df = pd.concat([df, row])
                    
    return df


print('computation_T5X3Y3_mean')
mean_time_space(5,3,3).to_csv('./result/OpenCV_NPZ_Compute/computation_T5X3Y3_mean.csv')

computation_T5X3Y3_mean
3Funny paddleboat GA.mp4_computeNPZ.npz ERROR: not working for some reason


Exception ignored in: <function NpzFile.__del__ at 0x000001FEFEAF8840>
Traceback (most recent call last):
  File "C:\Users\ramme\Anaconda3\lib\site-packages\numpy\lib\npyio.py", line 230, in __del__
    self.close()
  File "C:\Users\ramme\Anaconda3\lib\site-packages\numpy\lib\npyio.py", line 221, in close
    if self.zip is not None:
AttributeError: 'NpzFile' object has no attribute 'zip'


In [7]:
def mean_time_space(time_clusters=1, space_clusters_x=1, space_clusters_y=1):
    
    df = pd.DataFrame()

    # Bar update
    for file in files:
        # Load Completed File  
        try:
            data = np.load('./result/OpenCV_NPZ/'+file, allow_pickle=True)
        except:
            print(file, 'ERROR: not working for some reason')
            continue

        for selected_method in [0,1]:
            d=data['data_array'][selected_method]
            
            row = pd.DataFrame.from_dict({
                    'method':[data['data_method'][selected_method]],
                    'video':[file[:-15]]})

            time_bins      = int(d.shape[0]/time_clusters)
            space_bins_y   = int(d.shape[1]/space_clusters_x)
            space_bins_x   = int(d.shape[2]/space_clusters_y)

            # Split in time / space:
            for time in range(time_clusters):
                for space_x in range(space_clusters_x):
                    for space_y in range(space_clusters_y):
                        row['T'+str(time)+'X'+str(space_x)+'Y'+str(space_y)]= d[time*time_bins:(time+1)*time_bins,space_y*space_bins_y:(space_y+1)*space_bins_y, space_x*space_bins_x:(space_x+1)*space_bins_x].std() 
            df = pd.concat([df, row])
                    
    return df


print('computation_T5X3Y3_STD')
mean_time_space(5,3,3).to_csv('./result/OpenCV_NPZ_Compute/computation_T5X3Y3_STD.csv')

computation_T5X3Y3_STD
3Funny paddleboat GA.mp4_computeNPZ.npz ERROR: not working for some reason


Exception ignored in: <function NpzFile.__del__ at 0x000001FEFEAF8840>
Traceback (most recent call last):
  File "C:\Users\ramme\Anaconda3\lib\site-packages\numpy\lib\npyio.py", line 230, in __del__
    self.close()
  File "C:\Users\ramme\Anaconda3\lib\site-packages\numpy\lib\npyio.py", line 221, in close
    if self.zip is not None:
AttributeError: 'NpzFile' object has no attribute 'zip'
